In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install chardet

import chardet

# 파일의 인코딩 형식 확인
with open('/content/drive/MyDrive/ubion_project_2/감정 분류를 위한 대화 음성 데이터셋/4차년도.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)


In [ ]:
import zipfile
import os
import pandas as pd

# ZIP 파일 경로 및 압축 해제
zip_file_path = '/content/drive/MyDrive/ubion_project_2/감정 분류를 위한 대화 음성 데이터셋/4차년도.zip'
extract_path = '/content/wav_files'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)




In [ ]:
# 감지된 인코딩으로 CSV 파일 읽기
csv_file_path = '/content/drive/MyDrive/ubion_project_2/감정 분류를 위한 대화 음성 데이터셋/4차년도.csv'
encoding = 'CP949'  # chardet로 감지된 인코딩

data = pd.read_csv(csv_file_path, encoding=encoding)

# 'wav_id' 열을 사용하여 파일 경로 생성
data['file_path'] = data['wav_id'].apply(lambda x: os.path.join(extract_path, f"{x}.wav"))

# 로드된 데이터 확인
data.head()

In [ ]:
import soundfile as sf
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import os

# Wav2Vec2 모델 및 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
asr_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

# 음성 파일을 텍스트로 변환하는 함수
def speech_to_text(file_path):
    if not os.path.exists(file_path):
        return "File not found"  # Handle the case where the file doesn't exist

    try:
        speech, _ = sf.read(file_path)
        inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")
        input_values = inputs.input_values.to("cuda")
        with torch.no_grad():
            logits = asr_model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        return transcription[0]
    except Exception as e:
        return f"Error processing file: {e}"  # Catch and return any errors during processing

# 데이터프레임에 텍스트 데이터 추가
data['transcription'] = data['file_path'].apply(speech_to_text)

# Check for errors
print(data[data['transcription'].str.startswith("Error")])  # Print rows with errors
print(data[data['transcription'] == "File not found"])  # Print rows with missing files

In [ ]:
from transformers import BertTokenizer, BertModel

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to('cuda')


In [ ]:
import torch

def prepare_input(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding["input_ids"].squeeze().to('cuda')
    attention_mask = encoding["attention_mask"].squeeze().to('cuda')
    return input_ids.cpu().numpy(), attention_mask.cpu().numpy()

In [ ]:
data['input_ids'], data['attention_mask'] = zip(*data['transcription'].apply(prepare_input))

# 결과 확인
print(data[['file_path', 'transcription', 'input_ids', 'attention_mask']].head())


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to('cuda')

def prepare_input(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding["input_ids"].squeeze().cpu().numpy()
    attention_mask = encoding["attention_mask"].squeeze().cpu().numpy()
    return input_ids, attention_mask

data['input_ids'], data['attention_mask'] = zip(*data['transcription'].apply(prepare_input))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder를 사용하여 문자열 레이블을 정수로 변환
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['상황'])

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to('cuda')

def prepare_input(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding["input_ids"].squeeze().cpu().numpy()
    attention_mask = encoding["attention_mask"].squeeze().cpu().numpy()
    return input_ids, attention_mask

data['input_ids'], data['attention_mask'] = zip(*data['transcription'].apply(prepare_input))

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# BiLSTM 모델 정의
class BERT_BiLSTM(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim):
        super(BERT_BiLSTM, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # 2 for bidirectional

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_output, _ = self.lstm(bert_outputs.last_hidden_state)
        pooled_output = torch.mean(lstm_output, 1)
        return self.fc(pooled_output)

# 모델 인스턴스 생성
hidden_dim = 256
output_dim = len(data['label'].unique()) # 감정 레이블 개수
model = BERT_BiLSTM(bert_model, hidden_dim, output_dim).to('cuda')

# 데이터 준비
input_ids_list = [torch.tensor(x) for x in data['input_ids']]
attention_mask_list = [torch.tensor(x) for x in data['attention_mask']]

# 시퀀스 패딩
input_ids = nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True)
attention_mask = nn.utils.rnn.pad_sequence(attention_mask_list, batch_first=True)
labels = torch.tensor(data['label'].values).to('cuda')

# GPU로 데이터 이동
input_ids = input_ids.to('cuda')
attention_mask = attention_mask.to('cuda')

# 데이터셋 및 데이터로더 생성
dataset = TensorDataset(input_ids, attention_mask, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# 손실 함수 및 옵티마이저 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

# 모델 학습
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        b_input_ids, b_attention_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, b_attention_mask)
        loss = loss_fn(outputs, b_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1} Loss: {total_loss / len(train_dataloader)}')

    # 검증
    model.eval()
    eval_loss = 0
    for batch in val_dataloader:
        b_input_ids, b_attention_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, b_attention_mask)
        loss = loss_fn(outputs, b_labels)
        eval_loss += loss.item()
    print(f'Validation Loss: {eval_loss / len(val_dataloader)}')

In [ ]:
# 모델 가중치 저장
torch.save(model.state_dict(), 'bilstm_model.pth')

# LabelEncoder 클래스 저장
np.save('classes.npy', label_encoder.classes_)

In [ ]:
import os
import soundfile as sf
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, BertTokenizer, BertModel
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Wav2Vec2 모델 및 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
asr_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to('cuda')

# LabelEncoder 로드
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

# BiLSTM 모델 정의
class BERT_BiLSTM(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim):
        super(BERT_BiLSTM, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # 2 for bidirectional

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_output, _ = self.lstm(bert_outputs.last_hidden_state)
        pooled_output = torch.mean(lstm_output, 1)
        return self.fc(pooled_output)

# 모델 인스턴스 생성
hidden_dim = 256
output_dim = len(label_encoder.classes_) # 감정 레이블 개수
model = BERT_BiLSTM(bert_model, hidden_dim, output_dim).to('cuda')

# 학습된 모델 가중치 로드
model.load_state_dict(torch.load('bilstm_model.pth'))
model.eval()


In [ ]:
import os
import soundfile as sf
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, BertTokenizer, BertModel
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Wav2Vec2 모델 및 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
asr_model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to('cuda')

# LabelEncoder 로드
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

# BiLSTM 모델 정의
class BERT_BiLSTM(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim):
        super(BERT_BiLSTM, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # 2 for bidirectional

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_output, _ = self.lstm(bert_outputs.last_hidden_state)
        pooled_output = torch.mean(lstm_output, 1)
        return self.fc(pooled_output)

# 모델 인스턴스 생성
hidden_dim = 256
output_dim = len(label_encoder.classes_) # 감정 레이블 개수
model = BERT_BiLSTM(bert_model, hidden_dim, output_dim).to('cuda')

# 학습된 모델 가중치 로드
model.load_state_dict(torch.load('bilstm_model.pth'))
model.eval()

# 음성 파일을 텍스트로 변환하는 함수
def speech_to_text(file_path):
    try:
        if not os.path.exists(file_path):
            print(f"File does not exist: {file_path}")
            return ""
        speech, _ = sf.read(file_path)
        inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")
        input_values = inputs.input_values.to("cuda")
        with torch.no_grad():
            logits = asr_model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        return transcription[0]
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return ""

# 텍스트 데이터를 BERT 입력으로 변환하는 함수
def prepare_input(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding["input_ids"].squeeze().cpu().numpy()
    attention_mask = encoding["attention_mask"].squeeze().cpu().numpy()
    return input_ids, attention_mask

# 감정을 예측하는 함수
def predict_emotion(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predicted_label = torch.argmax(outputs, dim=1).cpu().numpy()
    return predicted_label

# 새로운 음성 파일 경로
new_file_path = '/content/sample_data/5ebd3dd7c38c123b9ec6deba.wav'

# Step 1: Load the new audio file and convert to text
transcription = speech_to_text(new_file_path)
print("Transcription:", transcription)

# Step 2: Convert text to BERT input format
input_ids, attention_mask = prepare_input(transcription)

# Step 3: Pad sequences and convert to tensor
input_ids = torch.tensor(input_ids).unsqueeze(0).to('cuda')
attention_mask = torch.tensor(attention_mask).unsqueeze(0).to('cuda')

# Step 4: Predict the emotion
predicted_label = predict_emotion(input_ids, attention_mask)
predicted_emotion = label_encoder.inverse_transform(predicted_label)
print("Predicted Emotion:", predicted_emotion[0])
